<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/tapi-logo-small.png" />

This notebook free for educational reuse under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/).

Created by [Melanie Walsh](https://melaniewalsh.org/) for the 2022 Text Analysis Pedagogy Institute, with support from the [National Endowment for the Humanities](https://neh.gov), [JSTOR Labs](https://labs.jstor.org/), and [University of Arizona Libraries](https://new.library.arizona.edu/).

For questions/comments/improvements, email melwalsh@uw.edu
____

# Working with Twitter Data (Lesson 2) — 6/22/2022

This is lesson **2** of 3 in the educational series on **Working with Twitter Data**. This notebook will demonstrate how researchers can download tweet collections that have been shared by other people and share their own tweets in ways that honor users' right to be forgotten while also adhering to Twitter's Terms of Service. 

**Audience:** Teachers / Learners / Researchers

**Use case:** Tutorial / How-To

**Difficulty:** Intermediate

**Completion time:** 30 minutes to 1 hour

**Knowledge Required/Recommended:** 

* Command line knowledge
* Python basics (variables, functions, lists, dictionaries)
* Pandas basics (Python library for data manipulation and analysis)


**Learning Objectives:**
After this lesson, learners will be able to:

1. Download tweet collections from a list of tweet IDs
2. Convert tweets to tweet IDs for sharing purposes

___

# Required Python Libraries
* [twarc2](https://twarc-project.readthedocs.io/en/latest/twarc2_en_us/) for collecting Twitter data.
* [plotly](https://plotly.com/python/) for making interactive plots 
* [pandas](https://pandas.pydata.org/) for manipulating and cleaning data

## Install Required Libraries

In [ ]:
### Install Libraries ###
!pip install twarc --upgrade
!pip install twarc-csv --upgrade
!pip install plotly

In [38]:
### Import Libraries ###
import plotly.express as px
import pandas as pd
# Set max column width
pd.options.display.max_colwidth = 400
# Set max number of columns
pd.options.display.max_columns = 95

# Twitter API Setup

*This lesson presumes that you've already installed and configured twarc, which was covered in [a previous lesson](Twitter-API-Setup).*

## Configure Twarc

Once twarc is installed, you need to configure it with your API keys and/or bearer token so that you can actually access the API. 

To configure twarc, you would typically run `twarc2 configure` from the command line. This will prompt twarc to ask for your bearer token, which you can copy and paste into the blank after the colon, and then press enter. You can optionally enter your API keys, as well.

<div class="admonition attention" name="html-admonition" style="background: orange; padding: 10px">
<p class="title">Note</p>
   To get your Bearer Token, go to your Twitter Developer portal: <a href= "https://developer.twitter.com/en/portal/dashboard">https://developer.twitter.com/en/portal/dashboard</a>

</div>

However, when working in a Jupyter notebook in the cloud, it is easiest to configure twarc and enter your Bearer Token in a single command. Please paste your Bearer Token between the quotations marks below and run the cell.

In [ ]:
#!printf '%s\n' "YOUR BEARER TOKEN HERE" "no" | twarc2 configure

Now you're ready to collect and analyze tweets!

## Tweet IDs

Twitter discourages developers and researchers from sharing full Twitter data openly on the web. They instead encourage developers and researchers to share *tweet IDs*:

> [If you provide Twitter Content to third parties, including downloadable datasets or via an API, you may only distribute **Tweet IDs**, Direct Message IDs, and/or User IDs.](https://developer.twitter.com/en/developer-terms/policy#4-e)

Tweet IDs are unique identifiers assigned to every tweet. They look like a random string of numbers: 1189206626135355397. Each tweet ID can be used to download the full data associated with that tweet (if the tweet still exists). This is a process called "hydration."

<img src="https://cdn.pixabay.com/photo/2013/07/12/19/24/sapling-154734_960_720.png" width=100% >

**Hydration: a young tweet ID sprouts into a full tweet (to be read in David Attenborough's voice)**

There are actually two reasons that you might want to dehydrate tweets and/or hydrate tweet IDs: first, to responsibly share Twitter data with others and/or access Twitter data shared by others; second, to get more information about the Twitter data that you yourself collected.

If you collected tweets in real time, for example, you collected those tweets immediately after they were published, which means that they will not contain any retweet or favorite count information. Nobody's had time to retweet them yet! So if you'd like to retroactively get retweet and favorite count information about your tweets, then you would want to dehydrate and rehydrate them.

## Hydrate Tweets

`twarc2 hydrate tweet_ids.txt > tweets.jsonl`

To transform a list of tweet IDs into full Twitter data, you can run the twarc command `twarc2 hydrate` with the name of your tweet IDs text file followed by the output operator `>` and the desired name of your JSONL file.

> tweet ID —> tweet = hydration <br>
> tweet ID <— tweet = dehydration

Kevin McElwee conducted a [fascinating analysis](https://www.kmcelwee.com/fortune-100-blm-report/site/index.html) of Fortune 100 companies' response to the #BlackLivesMatter protests following George Floyd's death in May 2020. He collected tweets from 91 of these companies' Twitter accounts between May 25 and July 25, 2020, and he then shared these tweets as [tweet IDs](https://github.com/kmcelwee/fortune-100-blm-dataset/blob/main/data/ids/fortune-100-tweet-ids.txt).

Let's re-hydrate the Twitter data that McElwee collected about these Fortune 100 companies.

In [1]:
!twarc2 hydrate fortune-100-tweet-ids.txt > fortune-100-tweets.jsonl

In [12]:
!twarc2 csv fortune-100-tweets.jsonl fortune-100-tweets.csv

100%|██████████████| Processed 17.6M/17.6M of input file [00:02<00:00, 6.99MB/s]

ℹ️
Parsed 6071 tweets objects from 62 lines in the input file.
Wrote 6071 rows and output 74 columns in the CSV.



Read in CSV file

In [14]:
tweets_df = pd.read_csv('fortune-100-tweets.csv', parse_dates = ['created_at'])

Let's apply the helper functions and create new columns.

In [15]:
# Find the type of tweet
def find_type(tweet):
    
    # Check to see if tweet contains retweet, quote tweet, or reply tweet info
    contains_retweet = tweet['referenced_tweets.retweeted.id']
    contains_quote = tweet['referenced_tweets.quoted.id']
    contains_reply = tweet['referenced_tweets.replied_to.id']
    
    # Does tweet contain retweet info? (Is this category not NA or empty?)
    if pd.notna(contains_retweet):
        return "retweet"
    # Does tweet contain quote and reply info?
    elif pd.notna(contains_quote) and pd.notna(contains_reply):
        return "quote/reply"
    # Does tweet contain quote info? 
    elif pd.notna(contains_quote):
        return "quote"
    # Does tweet contain reply info? 
    elif pd.notna(contains_reply):
        return "reply"
    # If it doesn't contain any of this info, it must be an original tweet
    else:
        return "original"

# Make Tweet URL
def make_tweet_url(tweets):
    # Get username
    username = tweets[0]
    # Get tweet ID
    tweet_id = tweets[1]
    # Make tweet URL
    tweet_url = f"https://twitter.com/{username}/status/{tweet_id}"
    return tweet_url

In [16]:
# Create tweet type column
tweets_df['type'] = tweets_df.apply(find_type, axis =1)
# Create tweet URL column
tweets_df['tweet_url'] = tweets_df[['author.username', 'id']].apply(make_tweet_url, axis='columns')

Let's select and rename only the columns we're interested in.

In [ ]:
# Select columns of interest
clean_tweets_df = tweets_df[['created_at', 'author.username', 'author.name', 'author.description',
                             'author.verified', 'type', 'text', 'public_metrics.retweet_count', 
                             'public_metrics.like_count', 'public_metrics.reply_count', 'public_metrics.quote_count',
                             'tweet_url', 'lang', 'source', 'geo.full_name']]

# Rename columns for convenience
clean_tweets_df = clean_tweets_df.rename(columns={'created_at': 'date', 'public_metrics.retweet_count': 'retweets', 
                          'author.username': 'username', 'author.name': 'name', 'author.verified': 'verified', 
                          'public_metrics.like_count': 'likes', 'public_metrics.quote_count': 'quotes', 
                          'public_metrics.reply_count': 'replies', 'author.description': 'user_bio'})

clean_tweets_df

### Sort By Retweets

What are the top retweeted tweets? We can use [pandas `sort_values()` method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) to sort any column in ascending or descending order.

In [ ]:
clean_tweets_df.sort_values(by="retweets", ascending=False)

### Count Tweets Per Category

How many tweets from each company are here?

In [ ]:
clean_tweets_df['name'].value_counts()

## Filter tweets by text

How many tweets contain the word "Black" as in "Black Americans" or "Black Lives Matter"? We can use [pandas `.str.contains()` method](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html) to search for specific text.

In [ ]:
text_filter = clean_tweets_df['text'].str.contains("Black")
clean_tweets_df[text_filter]

Kevin McElwee's analysis is also an excellent example of what's possible when you combine computational work with qualitative coding and hand tagging: https://www.kmcelwee.com/fortune-100-blm-report/site/corporate-summaries.htmlm

## Dehydrate Tweets

`twarc2 dehydrate tweets.jsonl > tweet_ids.txt`

To transform your Twitter own data into a list of tweet IDs (so that you can share your data openly on the web), you can run the twarc command `twarc2 dehydrate` with the name of your JSONL file followed by the output operator `>` and the desired name of your tweet ID text file.

> tweet ID —> tweet = hydration <br>
> tweet ID <— tweet = dehydration

<div class="admonition attention" name="html-admonition" style="background: lightyellow; padding: 10px">
<p class="title">Attention 🚨</p>
    Remember that the <code>--archive</code> flag and full-archive search functionality is only available to those who have an <a href= "https://developer.twitter.com/en/products/twitter-api/academic-research">Academic Research account</a>. 
    Students with Essential API access should remove the <code>--archive</code> flag from the code below.

</div>

In [24]:
!twarc2 search --archive "plums in the icebox is:verified" --limit 500 > tweets.jsonl

Let's dehydrate the Twitter data that we collected.

In [26]:
!twarc2 dehydrate tweets.jsonl > tweets.txt

ℹ️  Parsed 578 tweets IDs from 6 lines in tweets.jsonl file.


If we `open()` and `.read()` the tweet IDs file that we just created, it looks something like this:

In [ ]:
tweet_ids = open("tweets.txt", encoding="utf-8").read()
print(tweet_ids)

## Where to Find Tweet IDs

You can find repositories of tweet IDs that have been shared by other researchers in the following places:

- DocNow Catalog: https://catalog.docnow.io/

- George Washington University Tweet IDs: https://dataverse.harvard.edu/dataverse/gwu-libraries